# Домашнее задание к лекции «Понятие класса»

## Задание 1
Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

In [3]:
import requests

In [22]:
class Rate:
    def __init__(self, format_='value'):
        self.format = format_
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                return response[currency]['Value']
        
        return 'Error'
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')
    
    def max_course_currency(self):
        response = self.exchange_rates()
        max_value = 0
        max_name = None
        for currency in response.values():
            if currency['Value']>=max_value:
                max_value = currency['Value']
                max_name = currency['Name']
        return max_name

In [31]:
r = Rate(format_="full")
r.max_course_currency()

'СДР (специальные права заимствования)'

## Задание 2
Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [37]:
class Rate:
    def __init__(self, format_='value', diff=True):
        self.format = format_
        self.diff = diff
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']
    
    def make_format(self, currency, diff=True):
        """
        Возвращает информацию о валюте currency в трёх вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        - только значение:
        Rate('value').make_format('EUR')
        79.4966
        
        - разница между текущим значением и предыдущим:
        Rate('value').make_format('EUR', False)
        23.3435
        """
        self.diff = diff
        
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                if diff:
                    return response[currency]['Value']
                else:
                    return response[currency]['Value']-response[currency]['Previous']
                
        
        return 'Error'
    
    def eur(self, diff=True):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR', diff)
    
    def usd(self, diff=True):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD', diff)
    
    def brl(self, diff=True):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL', diff)
    
    def max_course_currency(self):
        response = self.exchange_rates()
        max_value = 0
        max_name = None
        for currency in response.values():
            if currency['Value']>=max_value:
                max_value = currency['Value']
                max_name = currency['Name']
        return max_name

In [40]:
r = Rate()
r.eur(diff=False)

-0.04760000000000275

## Задание 3
Напишите класс Designer, который учитывает количество международных премий. Подсказки в коде занятия в разделе “Домашнее задание задача 3”.

### Комментарий по классу Designer такой:
Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла”). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять).

Класс Designer пишется по аналогии с классом Developer из материалов занятия.

In [41]:
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)
    
    def check_if_it_is_time_for_upgrade(self):
        pass

In [42]:
class Developer(Employee):
    def __init__(self, name, seniority):
        super().__init__(name, seniority)
         
    def check_if_it_is_time_for_upgrade(self):
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все разработчики проходят аккредитацию
        self.seniority += 1
        
        # условие повышения сотрудника из презентации
        if self.seniority % 5 == 0:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()

In [69]:
class Designer(Employee):
    
    def __init__(self, name, seniority, awards=2):
        super().__init__(name, seniority)
        #по умолчанию - две международные премии
        #на каждую премию - по два пункта к seniority
        self.seniority+=2*awards
        
    def got_int_award(self):
        #плюс два пункта в случае получения международной премии
        self.seniority+=2
    
    def check_if_it_is_time_for_upgrade(self):
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все разработчики проходят аккредитацию
        self.seniority += 1
        
        # условие повышения сотрудника из презентации
        if self.seniority % 7 == 0:
            self.grade_up()
        
        print('Текущий уровень: ',self.seniority)
        # публикация результатов
        return self.publish_grade()

In [70]:
alex = Designer('Александр', 0)

In [71]:
for i in range(0,10):
    alex.check_if_it_is_time_for_upgrade()

Текущий уровень:  5
Александр 1
Текущий уровень:  6
Александр 1
Текущий уровень:  7
Александр 2
Текущий уровень:  8
Александр 2
Текущий уровень:  9
Александр 2
Текущий уровень:  10
Александр 2
Текущий уровень:  11
Александр 2
Текущий уровень:  12
Александр 2
Текущий уровень:  13
Александр 2
Текущий уровень:  14
Александр 3
